# Production Impact Analysis for Equipment Failures

This notebook demonstrates how to use NeqSim's Production Impact Analyzer to:

1. **Simulate equipment failures** - Trip, degraded operation, maintenance
2. **Compare production loss** vs. full plant shutdown
3. **Get optimized setpoints** for degraded operation
4. **Rank equipment by criticality** - identify most critical equipment

This is essential for:
- **Risk assessment** - Understanding impact of equipment failures
- **Maintenance planning** - Deciding when to do preventive maintenance
- **Operations optimization** - Running optimally during equipment outages

In [ ]:
# Import NeqSim Java classes
import jpype
import jpype.imports

# Start JVM if not already running
if not jpype.isJVMStarted():
    import neqsim
    
# Import Java classes
from neqsim.thermo.system import SystemSrkEos
from neqsim.process.equipment.stream import Stream
from neqsim.process.equipment.separator import Separator
from neqsim.process.equipment.compressor import Compressor
from neqsim.process.equipment.heatexchanger import Cooler
from neqsim.process.processmodel import ProcessSystem
from neqsim.process.util.optimizer import ProductionImpactAnalyzer, ProductionImpactResult
from neqsim.process.equipment.failure import EquipmentFailureMode

print("NeqSim Production Impact Analysis loaded successfully!")

## 1. Create a Gas Processing System

We'll create a typical offshore gas processing train with:
- HP Separator
- Two-stage compression with intercooling
- Export cooler

In [ ]:
# Create fluid composition (typical natural gas)
fluid = SystemSrkEos(298.15, 50.0)
fluid.addComponent("methane", 0.85)
fluid.addComponent("ethane", 0.08)
fluid.addComponent("propane", 0.04)
fluid.addComponent("n-butane", 0.02)
fluid.addComponent("CO2", 0.01)
fluid.setMixingRule("classic")

# Create feed stream
feed = Stream("Well Feed", fluid)
feed.setFlowRate(100000, "kg/hr")  # 100 tonnes/hr
feed.setTemperature(60, "C")
feed.setPressure(50, "bara")

# HP Separator
hp_sep = Separator("HP Separator", feed)
hp_sep.setInternalDiameter(2.5)

# Stage 1 Compression (50 -> 100 bara)
comp1 = Compressor("Stage 1 Compressor", hp_sep.getGasOutStream())
comp1.setOutletPressure(100.0, "bara")
comp1.setPolytropicEfficiency(0.76)

# Intercooler
cooler1 = Cooler("Intercooler", comp1.getOutletStream())
cooler1.setOutTemperature(35, "C")

# Stage 2 Compression (100 -> 180 bara)
comp2 = Compressor("Stage 2 Compressor", cooler1.getOutletStream())
comp2.setOutletPressure(180.0, "bara")
comp2.setPolytropicEfficiency(0.76)

# Export Cooler
export_cooler = Cooler("Export Cooler", comp2.getOutletStream())
export_cooler.setOutTemperature(40, "C")

# Build process system
process = ProcessSystem()
process.add(feed)
process.add(hp_sep)
process.add(comp1)
process.add(cooler1)
process.add(comp2)
process.add(export_cooler)

# Run baseline
process.run()

print("=== Baseline Process Conditions ===")
print(f"Feed Rate: {feed.getFlowRate('kg/hr'):.0f} kg/hr")
print(f"Export Rate: {export_cooler.getOutletStream().getFlowRate('kg/hr'):.0f} kg/hr")
print(f"Export Pressure: {export_cooler.getOutletStream().getPressure('bara'):.1f} bara")
print(f"Export Temperature: {export_cooler.getOutletStream().getTemperature('C'):.1f} °C")
print(f"Stage 1 Power: {comp1.getPower('kW'):.0f} kW")
print(f"Stage 2 Power: {comp2.getPower('kW'):.0f} kW")
print(f"Total Power: {comp1.getPower('kW') + comp2.getPower('kW'):.0f} kW")

## 2. Create Production Impact Analyzer

The analyzer will help us understand the impact of equipment failures.

In [ ]:
# Create the analyzer
analyzer = ProductionImpactAnalyzer(process)
analyzer.setFeedStreamName("Well Feed")
analyzer.setProductStreamName("Export Cooler")

# Optional: Set product price for economic calculations ($/kg)
# Assuming ~$5/MMBtu natural gas, converting to $/kg
analyzer.setProductPricePerKg(0.25)  # $0.25/kg

print("Production Impact Analyzer created")

## 3. Analyze Single Equipment Failure

Let's see what happens if Stage 1 Compressor trips.

In [ ]:
# Analyze Stage 1 Compressor failure
result = analyzer.analyzeFailureImpact("Stage 1 Compressor")

print("=== Stage 1 Compressor Trip Analysis ===")
print(f"Equipment: {result.getEquipmentName()}")
print(f"Failure Type: {result.getFailureMode().getName() if result.getFailureMode() else 'Unknown'}")
print()
print(f"Baseline Production: {result.getBaselineProductionRate():.0f} kg/hr")
print(f"Production with Failure: {result.getProductionWithFailure():.0f} kg/hr")
print(f"Production Loss: {result.getAbsoluteLoss():.0f} kg/hr ({result.getPercentLoss():.1f}%)")
print()
print(f"vs Full Shutdown: {result.getLossVsFullShutdown():+.0f} kg/hr")
print(f"  (positive = degraded operation is better than shutdown)")
print()
print(f"Economic Impact:")
print(f"  Loss per hour: ${result.getEconomicLossPerHour():.0f}")
print(f"  Loss per day: ${result.getEconomicLossPerDay():.0f}")
print()
print(f"Recommended Action: {result.getRecommendedAction()}")
print(f"Reason: {result.getRecommendationReason()}")

## 4. Compare Degraded Operation vs. Full Shutdown

Should we run with failed equipment or shut down completely?

In [ ]:
# Analyze with explicit comparison to plant stop
result = analyzer.compareToPlantStop("Intercooler")

print("=== Intercooler Failure: Run vs. Stop Analysis ===")
print(f"Baseline Production: {result.getBaselineProductionRate():.0f} kg/hr")
print(f"Production with Failure: {result.getProductionWithFailure():.0f} kg/hr")
print(f"Full Shutdown Production: {result.getFullShutdownProduction():.0f} kg/hr")
print()
print(f"Production Loss if Run Degraded: {result.getPercentLoss():.1f}%")
print(f"Production Loss if Shutdown: 100%")
print()
print(f"Recommendation: {result.getRecommendedAction()}")
print(f"Reason: {result.getRecommendationReason()}")
print()
recovery_time = result.getEstimatedRecoveryTime()
print(f"Estimated Recovery Time: {recovery_time:.1f} hours")
if result.getProductionWithFailure() > 0:
    saved_product = result.getProductionWithFailure() * recovery_time
    print(f"Product Saved by Running Degraded: {saved_product:.0f} kg")

## 5. Rank Equipment by Criticality

Which equipment would cause the most production loss if it fails?

In [ ]:
# Clear cache to ensure fresh analysis
analyzer.clearCache()

# Get criticality ranking
ranking = analyzer.rankEquipmentByCriticality()

print("=== Equipment Criticality Ranking ===")
print("(Ranked by production loss if equipment trips)")
print()
print(f"{'Rank':<6}{'Equipment':<25}{'Loss %':<10}{'Loss kg/hr':<12}{'Recommendation'}")
print("-" * 70)

for i, r in enumerate(ranking):
    print(f"{i+1:<6}{r.getEquipmentName():<25}{r.getPercentLoss():<10.1f}{r.getAbsoluteLoss():<12.0f}{r.getRecommendedAction()}")

## 6. Custom Failure Modes

Create custom failure scenarios like partial failures or degraded operation.

In [ ]:
# Create a degraded operation mode (50% capacity)
degraded_mode = EquipmentFailureMode.degraded(50)

print(f"Degraded Mode: {degraded_mode.getName()}")
print(f"  Type: {degraded_mode.getType()}")
print(f"  Capacity Factor: {degraded_mode.getCapacityFactor()*100:.0f}%")
print(f"  MTTR: {degraded_mode.getMttr():.1f} hours")
print()

# Create a custom failure mode using builder
custom_mode = (EquipmentFailureMode.builder()
    .name("Bearing Damage")
    .description("Compressor running on damaged bearing")
    .type(EquipmentFailureMode.FailureType.DEGRADED)
    .capacityFactor(0.7)  # 70% capacity
    .efficiencyFactor(0.85)  # Reduced efficiency
    .mttr(72.0)  # 3 days to repair
    .requiresImmediateAction(False)
    .build())

print(f"Custom Mode: {custom_mode.getName()}")
print(f"  Description: {custom_mode.getDescription()}")
print(f"  Capacity Factor: {custom_mode.getCapacityFactor()*100:.0f}%")
print(f"  Efficiency Factor: {custom_mode.getEfficiencyFactor()*100:.0f}%")
print(f"  MTTR: {custom_mode.getMttr():.1f} hours")

## 7. Direct Equipment Failure Simulation

You can also directly simulate failures on equipment objects.

In [ ]:
# Create a fresh process copy for this demo
test_process = process.copy()
test_comp = test_process.getUnit("Stage 1 Compressor")

print("=== Direct Equipment Failure Simulation ===")
print(f"Initial State:")
print(f"  Active: {test_comp.isActive()}")
print(f"  Failed: {test_comp.isFailed()}")
print(f"  Capacity Factor: {test_comp.getEffectiveCapacityFactor()*100:.0f}%")
print()

# Simulate a trip
test_comp.simulateTrip()
print("After simulateTrip():")
print(f"  Active: {test_comp.isActive()}")
print(f"  Failed: {test_comp.isFailed()}")
print(f"  Capacity Factor: {test_comp.getEffectiveCapacityFactor()*100:.0f}%")
print(f"  Failure Mode: {test_comp.getFailureMode()}")
print()

# Restore from failure
test_comp.restoreFromFailure()
print("After restoreFromFailure():")
print(f"  Active: {test_comp.isActive()}")
print(f"  Failed: {test_comp.isFailed()}")
print(f"  Capacity Factor: {test_comp.getEffectiveCapacityFactor()*100:.0f}%")
print()

# Simulate degraded operation
test_comp.simulateDegradedOperation(60)  # 60% capacity
print("After simulateDegradedOperation(60):")
print(f"  Active: {test_comp.isActive()}")
print(f"  Failed: {test_comp.isFailed()}")
print(f"  Capacity Factor: {test_comp.getEffectiveCapacityFactor()*100:.0f}%")

## 8. Multiple Equipment Failures

What if multiple pieces of equipment fail simultaneously?

In [ ]:
from java.util import Arrays

# Analyze multiple simultaneous failures
failed_equipment = Arrays.asList(["Stage 1 Compressor", "Intercooler"])
result = analyzer.analyzeMultipleFailures(failed_equipment)

print("=== Multiple Equipment Failure Analysis ===")
print(f"Failed Equipment: {result.getEquipmentName()}")
print(f"Baseline Production: {result.getBaselineProductionRate():.0f} kg/hr")
print(f"Production with Failures: {result.getProductionWithFailure():.0f} kg/hr")
print(f"Total Loss: {result.getPercentLoss():.1f}%")
print(f"Recommendation: {result.getRecommendedAction()}")

## 9. Get Optimized Setpoints

The analyzer can recommend optimized operating conditions during degraded operation.

In [ ]:
# Ensure optimization is enabled
analyzer.setOptimizeDegradedOperation(True)
analyzer.clearCache()

# Analyze with optimization
result = analyzer.analyzeFailureImpact("Stage 2 Compressor")

print("=== Optimized Setpoints for Degraded Operation ===")
print(f"Equipment Failed: {result.getEquipmentName()}")
print(f"Production with Failure: {result.getProductionWithFailure():.0f} kg/hr")
print(f"Optimized Production: {result.getOptimizedProductionWithFailure():.0f} kg/hr")
print()
print("Recommended Setpoints:")
setpoints = result.getOptimizedSetpoints()
for key in setpoints.keySet():
    print(f"  {key}: {setpoints.get(key):.2f}")

## 10. Export Results to JSON

Results can be exported to JSON for further analysis or reporting.

In [ ]:
import json

# Get JSON output
json_str = result.toJson()

# Parse and pretty-print
data = json.loads(str(json_str))
print(json.dumps(data, indent=2))

## Summary

The Production Impact Analyzer provides:

1. **Single Equipment Failure Analysis** - `analyzeFailureImpact(equipmentName)`
2. **Comparison to Shutdown** - `compareToPlantStop(equipmentName)`
3. **Criticality Ranking** - `rankEquipmentByCriticality()`
4. **Multiple Failures** - `analyzeMultipleFailures(equipmentList)`
5. **Optimized Setpoints** - Automatically calculated when `setOptimizeDegradedOperation(True)`

### Key Failure Mode Types

- `EquipmentFailureMode.trip(type)` - Complete equipment stop
- `EquipmentFailureMode.degraded(percent)` - Reduced capacity
- `EquipmentFailureMode.maintenance(hours)` - Planned shutdown
- `EquipmentFailureMode.bypassed()` - Equipment bypassed

### Direct Equipment Methods

- `equipment.simulateTrip()` - Simulate trip
- `equipment.simulateDegradedOperation(percent)` - Simulate reduced capacity
- `equipment.restoreFromFailure()` - Restore to normal
- `equipment.isFailed()` - Check if failed
- `equipment.getEffectiveCapacityFactor()` - Get current capacity